<a href="https://colab.research.google.com/github/Srini-UK/GenAI_Assignments/blob/main/LangChain_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install LangChain

In [ ]:
!pip install langchain google-generativeai msal requests

# Define Gemini Tool

In [ ]:
from langchain.tools import tool
import google.generativeai as genai
import json

genai.configure(api_key="YOUR_GEMINI_API_KEY")

@tool
def ask_gemini(question: str, metadata: str) -> str:
    """Use Gemini to answer a BI question based on metadata and sample data."""
    model = genai.GenerativeModel("gemini-2.5-flash")
    prompt = f"""
    You are a BI assistant. Answer the question using the metadata below.

    Question: {question}
    Metadata: {metadata}
    """
    return model.generate_content(prompt).text.strip()

# Define Power BI Tool

In [ ]:
@tool
def fetch_powerbi_metadata(report_id: str, workspace_id: str, access_token: str) -> str:
    """Fetch Power BI report metadata and sample data."""
    headers = {"Authorization": f"Bearer {access_token}"}
    url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports/{report_id}"
    report_info = requests.get(url, headers=headers).json()
    dataset_id = report_info.get("datasetId")
    sample_data = {}

    if dataset_id:
        tables_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/tables"
        tables_response = requests.get(tables_url, headers=headers).json()
        for table in [t["name"] for t in tables_response.get("value", [])][:2]:
            rows_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/tables/{table}/rows"
            rows_response = requests.get(rows_url, headers=headers).json()
            sample_data[table] = rows_response.get("value", [])[:5]

    return json.dumps({"report": report_info, "sample": sample_data})

# Build LangChain Agent

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key="YOUR_GEMINI_API_KEY")

tools = [ask_gemini, fetch_powerbi_metadata]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run Agent

In [ ]:
# Replace with your actual values
ACCESS_TOKEN = "your-access-token"
WORKSPACE_ID = "your-workspace-id"
REPORT_ID = "your-report-id"

response = agent.run(f"""
Fetch Power BI metadata for report {REPORT_ID} in workspace {WORKSPACE_ID} using access token {ACCESS_TOKEN},
then answer: What was the IT spend for BU Support in January 2014 for the USA region?
""")

print(response)